<a href="https://colab.research.google.com/github/Madhu-712/Madhu-712/blob/main/Gemini_Langchainagents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2023 Google LLC

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Setup

In [ ]:
!pip install -U -q "google-generativeai>=0.8.2"

In [ ]:
# import necessary modules.
import base64
import copy
import json
import pathlib
import requests


import PIL.Image
import IPython.display
from IPython.display import Markdown

try:
    # The SDK will automatically read it from the GOOGLE_API_KEY environment variable.
    # In Colab get the key from Colab-secrets ("🔑" in the left panel).
    import os
    from google.colab import userdata

    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
except ImportError:
    pass

import google.generativeai as genai

# Parse the arguments

model = 'gemini-1.5-pro' # @param {isTemplate: true}
contents_b64 = '...' # @param {isTemplate: true}
generation_config_b64 = 'eyJ0ZW1wZXJhdHVyZSI6MSwidG9wX3AiOjAuOTUsInRvcF9rIjo0MCwibWF4X291dHB1dF90b2tlbnMiOjgxOTJ9' # @param {isTemplate: true}
safety_settings_b64 = "e30="  # @param {isTemplate: true}

gais_contents = json.loads(base64.b64decode(contents_b64))

generation_config = json.loads(base64.b64decode(generation_config_b64))
safety_settings = json.loads(base64.b64decode(safety_settings_b64))

stream = False

# Convert and upload the files

tempfiles = pathlib.Path(f"tempfiles")
tempfiles.mkdir(parents=True, exist_ok=True)


drive = None
def upload_file_data(file_data, index):
    """Upload files to the Files API.

    For each file, Google AI Studio either sent:
    - a Google Drive ID,
    - a URL,
    - a file path, or
    - The raw bytes (`inline_data`).

    The API only understands `inline_data` or it's Files API.
    This code, uploads files to the files API where the API can access them.
    """

    mime_type = file_data["mime_type"]
    if drive_id := file_data.pop("drive_id", None):
        if drive is None:
          from google.colab import drive
          drive.mount("/gdrive")

        path = next(
            pathlib.Path(f"/gdrive/.shortcut-targets-by-id/{drive_id}").glob("*")
        )
        print("Uploading:", str(path))
        file_info = genai.upload_file(path=path, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if url := file_data.pop("url", None):
        response = requests.get(url)
        data = response.content
        name = url.split("/")[-1]
        path = tempfiles / str(index)
        path.write_bytes(data)
        print("Uploading:", url)
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if name := file_data.get("filename", None):
        if not pathlib.Path(name).exists():
            raise IOError(
                f"local file: `{name}` does not exist. You can upload files "
                'to Colab using the file manager ("📁 Files" in the left '
                "toolbar)"
            )
        file_info = genai.upload_file(path, display_name=name, mime_type=mime_type)
        file_data["file_uri"] = file_info.uri
        return

    if "inline_data" in file_data:
        return

    raise ValueError("Either `drive_id`, `url` or `inline_data` must be provided.")


contents = copy.deepcopy(gais_contents)

index = 0
for content in contents:
    for n, part in enumerate(content["parts"]):
        if file_data := part.get("file_data", None):
            upload_file_data(file_data, index)
            index += 1

import json
print(json.dumps(contents, indent=4))

## Call `generate_content`

In [ ]:
from IPython.display import display
from IPython.display import Markdown

# Call the model and print the response.
gemini = genai.GenerativeModel(model_name=model)

response = gemini.generate_content(
    contents,
    generation_config=generation_config,
    safety_settings=safety_settings,
    stream=stream,
)

display(Markdown(response.text))

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />Docs on ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google-gemini/cookbook/blob/main/quickstarts"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />More notebooks in the Cookbook</a>
  </td>
</table>

## [optional] Show the conversation

This section displays the conversation received from Google AI Studio.

In [ ]:
pip install -qU langchain-google-vertexai

In [ ]:

# Ensure your VertexAI credentials are configured

from langchain_google_vertexai import ChatVertexAI

llm = ChatVertexAI(model="gemini-1.5-flash")

#langchain Agents

In [ ]:

from datetime import datetime, timedelta
import pytz
import os
import requests
from langchain.agents import AgentExecutor, create_react_agent
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool
from langchain.prompts import PromptTemplate
from langchain.google_serp import GoogleSerpAPIWrapper


AERODAT_BASE_URL = "https://aerodat.flightaware.com/aerodat"
AERODAT_KEY = os.getenv("AERODAT_KEY")

def get_api_session():
    session = requests.Session()
    session.headers.update({"x-apikey": AERODAT_KEY})
    return session

def fetch_flight_data(flight_id, session):
    if " " in flight_id:
        flight_id = flight_id.split(" ")[1]

    start_date = (datetime.now().date() - timedelta(days=1)).strftime("%Y-%m-%d")
    end_date = datetime.now().date().strftime("%Y-%m-%d")

    api_resource = f"/flights?flightId={flight_id}&startDate={start_date}&endDate={end_date}"

    response = session.get(AERODAT_BASE_URL + api_resource)
    response.raise_for_status() # Raises HTTPError, if one occurred
    return response.json()['flights'][0]


def utc_to_local(utc_datetime_str, local_timezone_str):
    local_timezone = pytz.timezone(local_timezone_str)
    utc_datetime = datetime.strptime(utc_datetime_str, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=pytz.utc)
    local_datetime = utc_datetime.astimezone(local_timezone)
    return local_datetime.strftime("%Y-%m-%d %H:%M:%S")

In [ ]:
@tool
def get_flight_status(flight_id:str):
     """Returns Flight Information."""
      session = get_api_session()
        flight_data = fetch_flight_data(flight_id, session)


        dep_key = "estimatedOut" if "estimatedOut" in flight_data and flight_data["estimatedOut"] else \
                   "actualOut" if "actualOut" in flight_data and flight_data["actualOut"] else \
                  "scheduledOut"


        arr_key = "estimatedIn" if "estimatedIn" in flight_data and flight_data["estimatedIn"] else \
                   "actualIn" if "actualIn" in flight_data and flight_data["actualIn"] else \
                  "scheduledIn"

        flight_details = {
            "source": flight_data["origin"]["city"],
            "destination" : flight_data["destination"]["city"],
            "depart_time": utc_to_local(flight_data.get(dep_key), flight_data["origin"]["timezone"]),
            "arrival_time": utc_to_local(flight_data.get(arr_key), flight_data["destination"]["timezone"]),
            "status": flight_data["status"]
        }


        return f"""The current status of flight {flight_id} from {flight_details["source"]} to {flight_details["destination"]} is {flight_details["status"]} with the departure time as {flight_details["depart_time"]} and arrival time as {flight_details["arrival_time"]}"""

In [ ]:
tools =GetFlightStatus()

In [ ]:
prompt = PromptTemplate(
    template="""Answer the following questions as best as you can. You have access to the following tools:{tool_names}

Use the following format:

```tool_code
{agent_scratchpad}```""",
    input_variables=["input", "tool_names", "agent_scratchpad"],
)

In [ ]:
prompt

In [ ]:
llm = ChatVertexAI(model="gemini-1.5-flash")

In [ ]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [ ]:
Output=agent_executor.invoke({"input":"what is the status of EK24 and also include source and destination along with the timings"},verbose=True)

In [ ]:
response(Output)

In [ ]:
Output=agent_executor.invoke({"input":"If I am arriving Hyderabad by EK24 and it takes about 1hr to get luggages and getting home takes 2 hrs from airport. What time I should book a cab and when will I reach Home?},verbose=True)

In [ ]:
response(Output)